<a href="https://colab.research.google.com/github/djidachahrazed/Semantic-Similarity/blob/master/imdb_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PACKAGES 
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
#QUESTION1:
dataFrm = pd.read_csv(r"/content/IMDB.csv")  
dataFrm.columns= ["review", "sentiment"]
print(dataFrm.head())

In [ ]:
# QUESTION 2:
#*************QUESTION 2-1**************
print(dataFrm.sentiment.value_counts())

In [ ]:
#************QUESTION 2-2**************
dataFrm["phrase_len"] = [len(x) for x in dataFrm.review]
print(dataFrm.head(2))

In [ ]:
#************QUESTION 2-3***************
group = dataFrm.groupby("sentiment")
#mean: moyenne
print("Taille moyenne pour neg. reviews: ", group.get_group("negative")["phrase_len"].mean())
print("Taille moyenne pour pos. reviews: ", group.get_group("positive")["phrase_len"].mean())

In [ ]:
#************QUESTION 2-4***************
#loc : Access a group of rows and columns by label(s) or a boolean array.
dataFrm = dataFrm.loc[ (dataFrm["phrase_len"] > 3) , : ]
print(dataFrm.head(5))

In [ ]:
#************QUESTION 2-5***************
dataFrm = dataFrm.drop(columns = "phrase_len" )
print(dataFrm.head(2))

In [ ]:
#Question3
#------------QUESTION3.1: netoyage de corpus ----------
# upper to lower
def wordLowerCase(text):
    return text.lower()
def tokenization(text):
    return (text.split())
def stopword_elimination(text):
    listes_words=tokenization(text)
    new_list=[]
    for word in listes_words:
        if word not in stopwords.words("english"):
            new_list.append(word)
    return new_list
def stemming(text):
    port=PorterStemmer()
    print("\nstemming")
    return " ".join([port.stem(i)for i in text.split()])

In [ ]:
dataFrm["review"] = dataFrm["review"].str.lower().str.split()
print(dataFrm)

In [ ]:
nltk.download('stopwords')

In [ ]:
stop = stopwords.words('english')
dataFrm["review"].apply(lambda x: [item for item in x if item not in stop])
print(dataFrm.head(10))


In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
  return " ".join([stemmer.stem(word) for word in text])

dataFrm["review"] =dataFrm["review"].apply(lambda text: stem_words(text))
print(dataFrm["review"].head(10))

In [ ]:
nltk.download('punkt')

In [ ]:
#QUESTION4
#tf_idf
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
text_count_matrix = tfidf.fit_transform(dataFrm.review)
#print(text_count_matrix)

In [ ]:
#QUESTION5
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_count_matrix, dataFrm.sentiment, test_size=0.30, random_state=2)

In [ ]:
#QUESTION6
# let's use Naive Bayes classifier and fit our model:
from sklearn.naive_bayes import MultinomialNB 
MNB = MultinomialNB()
MNB.fit(x_train, y_train)
#4. Evaluating the model
from sklearn import metrics
accuracy_score = metrics.accuracy_score(MNB.predict(x_test), y_test)
print("accuracy_score = " + str('{:04.2f}'.format(accuracy_score*100))+" %")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report: \n", classification_report(y_test, MNB.predict(x_test),target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(y_test, MNB.predict(x_test)))